In [41]:
import pyrealsense2 as rs
import numpy as np
import cv2

import time
import math

import pygame
from pygame.locals import *

from OpenGL.GL import *
from OpenGL.GLU import *

In [42]:
deviceDetect = False
ctx = rs.context()
ds5_dev = rs.device()
devices = ctx.query_devices()

for dev in devices:
    productName = str(dev.get_info(rs.camera_info.product_id))
  
    # DS 435 config
    if productName in "0B07":
        EnableColorStream = True
        EnableInfraredStream = False
        EnableDepthStream = True
        deviceDetect = True
        break
    
    # DS 415 config
    elif productName in "0AD3":
        EnableColorStream = True
        EnableInfraredStream = False
        EnableDepthStream = True
        deviceDetect = True
        break
        
    # DS 410 config
    elif productName in "0AD2":
        EnableColorStream = False
        EnableInfraredStream = True
        EnableDepthStream = True
        deviceDetect = True
        break

if deviceDetect is not True:
    raise Exception("No supported device was found")

In [43]:
ColorStreamResWidth = 1280
ColorStreamResHeight = 720
ColorStreamFPS = 30

InfraredStreamResWidth = 1280
InfraredStreamResHeight = 720
InfraredStreamFPS = 30

DepthStreamResWidth = 1280
DepthStreamResHeight = 720
DepthStreamFPS = 30
DepthEqualization = True
DepthMin = 100
DepthMax = 1000 # 1000 = 1 meter


font = cv2.FONT_HERSHEY_TRIPLEX
fontA = cv2.FONT_HERSHEY_DUPLEX
fontB = cv2.FONT_HERSHEY_SIMPLEX

textString = ''
sizeLine = 3
sizeLineP = 1
colorBlack = (0, 0, 0)
colorBlue = (255, 0, 0)
colorGreen = (0, 255, 0)
colorRed = (0, 0, 255)

colorYellow = (0, 255,255)
colorPurple = (255, 0,255)
colorTeal = (255,255,0)

colorWhite = (255,255,255)

In [44]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream is True:
    config.enable_stream(
        rs.stream.color, ColorStreamResWidth, ColorStreamResHeight, rs.format.bgr8, ColorStreamFPS)

if EnableInfraredStream is True:
    config.enable_stream(
        rs.stream.infrared, InfraredStreamResWidth, InfraredStreamResHeight, rs.format.bgr8, InfraredStreamFPS)
    
if EnableDepthStream is True:
    config.enable_stream(
        rs.stream.depth, DepthStreamResWidth, DepthStreamResHeight, rs.format.z16, DepthStreamFPS)

# Start streaming
cfg = pipeline.start(config)

# Alignment
if EnableColorStream is True:
    align_to = rs.stream.color
elif EnableInfraredStream is True:
    align_to = rs.stream.infrared

align = rs.align(align_to)
    
# Advanced settings
dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 4)
depth_sensor.set_option(rs.option.enable_auto_exposure, 1)
#depth_sensor.set_option(rs.option.min_distance, DepthMin)
#depth_sensor.set_option(rs.option.max_distance, DepthMax)

# Get depth scale
scale = depth_sensor.get_depth_scale()
print("depth scale:" + str(scale))

# PointCloud settings
#pc = rs.pointcloud()

# Get intrinsics
if EnableColorStream is True:
    stream = cfg.get_stream(rs.stream.color )
    profile = stream.as_video_stream_profile()
elif EnableInfraredStream is True:
    stream = cfg.get_stream(rs.stream.infrared)
    profile = stream.as_video_stream_profile()
    
intrin = profile.get_intrinsics()
print("Intrinsics:\n  width:" , intrin.width, "\n  height:" , intrin.height, "\n  ppx:" , intrin.ppx, "\n  ppy:" , intrin.ppy, "\n  fx:" , intrin.fx, "\n  fy:" , intrin.fy, "\n  model:" , intrin.model)

depth scale:0.0010000000474974513
Intrinsics:
  width: 1280 
  height: 720 
  ppx: 650.6104736328125 
  ppy: 346.4551696777344 
  fx: 915.43896484375 
  fy: 914.4653930664062 
  model: distortion.brown_conrady


In [45]:
def procColorMap(input_image):
    inp = input_image.copy()
    
    
    if DepthEqualization:
        scaleAlpha = 255 / (DepthMax - DepthMin)
        inp = cv2.convertScaleAbs(inp, None, scaleAlpha, -1*DepthMin*scaleAlpha)
        mask = np.where((input_image == 0), 0, 1).astype('uint8')
        inp = inp * mask[:,:]
        
        '''
        height, width = inp.shape
        for i in range(0, width):
            for j in range(0, height):
                if inp[j,i] is not 0:
                    inp[j,i] = int((inp[j,i] - DepthMin) * scaleAlpha)
        '''
    else:
        scaleAlpha = 255 / DepthMax
        inp = cv2.convertScaleAbs(inp, None, scaleAlpha, 0)
        
    depth_colormap = cv2.applyColorMap(inp, cv2.COLORMAP_JET)
    
    return depth_colormap

In [46]:
def pixelToPoint(pixel, intrin, depth):
    point = []
        
    x = float((pixel[0] - intrin.ppx) / intrin.fx)
    y = float((pixel[1] - intrin.ppy) / intrin.fy)
    
    point.append(float(depth * x))
    point.append(float(depth * y))
    point.append(float(depth))
   
    return point

In [47]:
# utility global parameters
utilityCoordX = -1
utilityCoordY = -1

flagCoord = True
flagLine = True
flagCapture = False

if EnableColorStream is True:
    CoordBorderW = ColorStreamResWidth
    CoordBorderH = ColorStreamResHeight
elif EnableInfraredStream is True:
    CoordBorderW = InfraredStreamResWidth
    CoordBorderH = InfraredStreamResHeight

In [48]:
# proc for development
def procDepthValue(event,x,y,flags,param):
    global utilityCoordX, utilityCoordY
    
    if event is cv2.EVENT_MOUSEMOVE:
        if x != 0 and y != 0:
            utilityCoordX = x
            utilityCoordY = y

In [49]:
if EnableColorStream is True:
    cv2.namedWindow('Color image')
    cv2.setMouseCallback('Color image', procDepthValue)

if EnableInfraredStream is True:
    cv2.namedWindow('Infrared image')
    cv2.setMouseCallback('Infrared image', procDepthValue) 
    
if EnableDepthStream is True:
    cv2.namedWindow('Depth image')
    cv2.setMouseCallback('Depth image', procDepthValue)

In [50]:
# draw line function
def procDrawLine():
    if flagLine is True:
        global color_image, infrared_image, depth_colormap

        if EnableColorStream is True:
            cv2.line(color_image, (0, int(ColorStreamResHeight/2)), (ColorStreamResWidth, int(ColorStreamResHeight/2)), colorGreen, 1)
            cv2.line(color_image, (int(ColorStreamResWidth/2), 0), (int(ColorStreamResWidth/2), ColorStreamResHeight), colorGreen, 1)

        if EnableInfraredStream is True:
            cv2.line(infrared_image, (0, int(ColorStreamResHeight/2)), (ColorStreamResWidth, int(ColorStreamResHeight/2)), colorGreen, 1)
            cv2.line(infrared_image, (int(ColorStreamResWidth/2), 0), (int(ColorStreamResWidth/2), ColorStreamResHeight), colorGreen, 1)

        if EnableDepthStream is True:
            cv2.line(depth_colormap, (0, int(ColorStreamResHeight/2)), (ColorStreamResWidth, int(ColorStreamResHeight/2)), colorGreen, 1)
            cv2.line(depth_colormap, (int(ColorStreamResWidth/2), 0), (int(ColorStreamResWidth/2), ColorStreamResHeight), colorGreen, 1)


In [51]:
flagPoint = False
eventQuit = False
pc = rs.pointcloud()
points = rs.points()

pygame.init()
display = (800,600)
pygame.display.set_mode(display, DOUBLEBUF|OPENGL)

gluPerspective(45, (display[0]/display[1]), 0.1, 50.0)

glTranslatef(0.0,0.0, -5)

In [52]:
def procPointCloud():
    global flagPoint
    global color_frame, depth_frame
    
    if flagPoint is True:
        pc.map_to(color_frame)
        points = pc.calculate(depth_frame)
        
        verticies = np.asanyarray(points.get_vertices())
        
        glBegin(GL_POINTS)
        for vertex in verticies:
            glVertex3fv(vertex)
            
        glEnd()
        flagPoint = False

In [53]:
try:
    while True:
        tick1 = cv2.getTickCount()
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()

        # Aligh frames
        aligned_frames = align.proccess(frames)

        if EnableColorStream is True:
            color_frame = aligned_frames.get_color_frame()
            if not color_frame:
                continue

        if EnableInfraredStream is True:
            infrared_frame = aligned_frames.first(rs.stream.infrared)
            if not infrared_frame:
                continue

        if EnableDepthStream is True:
            depth_frame = aligned_frames.get_depth_frame()
            if not depth_frame:
                continue

        # Convert images to numpy arrays
        if EnableColorStream is True:
            color_image = np.asanyarray(color_frame.get_data())

        if EnableInfraredStream is True:
            infrared_image = np.asanyarray(infrared_frame.get_data())

        if EnableDepthStream is True:
            depth_image = np.asanyarray(depth_frame.get_data())

        # colorize depth image
        depth_colormap = procColorMap(depth_image)
                  
        # draw line function
        procDrawLine()
        
        # FPS calculate
        tick2 = cv2.getTickCount()
        tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

        # Show images
        if EnableColorStream is True:
            temp = color_image.copy()

            if utilityCoordX <= CoordBorderW and utilityCoordY <= CoordBorderH and flagCoord is True:
                cv2.circle(temp, (utilityCoordX,utilityCoordY), 5, colorWhite, -1)
                
                point = pixelToPoint([utilityCoordX, utilityCoordY], intrin, depth_image[utilityCoordY, utilityCoordX])
                textString = str(math.floor(point[1])) + " " + str(math.floor(point[0])) + " " + str(math.floor(point[2]))
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY+40), fontA, 1, colorBlack, 1, cv2.LINE_AA)
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY+40), fontB, 1, colorYellow, 1, cv2.LINE_AA)

            temp = cv2.copyMakeBorder(temp, 0, 40, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            textString = str(tick) + " ms"
            cv2.putText(temp, textString, (10, ColorStreamResHeight+30), font, 1, colorTeal, 1, cv2.LINE_AA)

            cv2.imshow('Color image', temp)
            
            if flagCapture is True and flagCoord is True:
                fileName = "../imageCapture/colorText_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
                cv2.imwrite(fileName, temp, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        if EnableInfraredStream is True:
            temp = infrared_image.copy()

            if utilityCoordX <= CoordBorderW and utilityCoordY <= CoordBorderH and flagCoord is True:
                cv2.circle(temp, (utilityCoordX,utilityCoordY), 5, colorWhite, -1)
              
                point = pixelToPoint([utilityCoordX, utilityCoordY], intrin, depth_image[utilityCoordY, utilityCoordX])
                textString = str(math.floor(point[1])) + " " + str(math.floor(point[0])) + " " + str(math.floor(point[2]))
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY+40), fontA, 1, colorBlack, 1, cv2.LINE_AA)
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY+40), fontB, 1, colorYellow, 1, cv2.LINE_AA)

            temp = cv2.copyMakeBorder(temp, 0, 40, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            textString = str(tick) + " ms"
            cv2.putText(temp, textString, (10, InfraredStreamResHeight+30), font, 1, colorTeal, 1, cv2.LINE_AA)

            cv2.imshow('Infrared image', temp)
            
            if flagCapture is True and flagCoord is True:
                fileName = "../imageCapture/infraredText_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
                cv2.imwrite(fileName, temp, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        if EnableDepthStream is True:
            temp = depth_colormap.copy()

            if utilityCoordX <= CoordBorderW and utilityCoordY <= CoordBorderH and flagCoord is True:
                cv2.circle(temp, (utilityCoordX,utilityCoordY), 5, colorWhite, -1)
                
                point = pixelToPoint([utilityCoordX, utilityCoordY], intrin, depth_image[utilityCoordY, utilityCoordX])
                textString = str(math.floor(point[1])) + " " + str(math.floor(point[0])) + " " + str(math.floor(point[2]))
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY+40), fontA, 1, colorBlack, 1, cv2.LINE_AA)
                cv2.putText(temp, textString, (utilityCoordX-25, utilityCoordY+40), fontB, 1, colorYellow, 1, cv2.LINE_AA)

            cv2.imshow('Depth image', temp)
            if flagCapture is True and flagCoord is True:
                fileName = "../imageCapture/depthText_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
                cv2.imwrite(fileName, temp, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        
        # Generate point cloud
        #procPointCloud()
                       
        pygame.display.flip()
        pygame.time.wait(10)

        # Keyboard command
        getKey = cv2.waitKey(10) & 0xFF
        if getKey is ord('c') or getKey is ord('C'):
            flagCapture = True
            pass
        elif getKey is ord('p') or getKey is ord('P'):
            flagPoint = True
            procPointCloud()
        elif getKey is ord('q') or getKey is ord('Q'):
            break
        elif getKey is ord('f') or getKey is ord('F'):
            if flagCoord is True:
                flagCoord = False
            else:
                flagCoord = True
        elif getKey is ord('l') or getKey is ord('L'):
            if flagLine is True:
                flagLine = False
            else:
                flagLine = True
        elif getKey is ord('x') or getKey is ord('X'):
            cv2.destroyAllWindows()

            if EnableColorStream is True:
                cv2.namedWindow('Color image')
                cv2.setMouseCallback('Color image', procDepthValue)

            if EnableInfraredStream is True:
                cv2.namedWindow('Infrared image')
                cv2.setMouseCallback('Infrared image', procDepthValue) 

            if EnableDepthStream is True:
                cv2.namedWindow('Depth image')
                cv2.setMouseCallback('Depth image', procDepthValue)

except Exception as e:
    print(e)
    pass

finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipeline.stop()
    pygame.quit()

("No array-type handler for type <class 'numpy.void'> (value: (-0., -0.,  0.)) registered", None)
